In [1]:
import os
import json
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nltk
import seaborn as sns ,sklearn,gensim
from nltk.corpus import stopwords
from collections import  Counter
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
with open('AQAD-master/AQQAD 1.0/FINAL_AAQAD-v1.0.json','rb') as f:
  data = json.load(f)


for allData in data['data']:
    print(allData['title']+"\n")
    print(allData['paragraphs'])
    break

بيونسيه

[{'qas': [{'question': 'متى بدأت بيونسي تصبح شعبية؟', 'id': 1, 'answers': [{'text': 'في أواخر التسعينات', 'answer_start': 220}], 'is_impossible': False}, {'question': 'ما هي المجالات التي تنافس عليها بيونسيه عندما كانت تكبر؟', 'id': 2, 'answers': [{'text': 'غناء ورقص', 'answer_start': 173}], 'is_impossible': False}, {'question': 'متى غادرت بيونسي طفل القدر وتصبح مغنية منفردة؟', 'id': 3, 'answers': [{'text': '(2003)،', 'answer_start': 515}], 'is_impossible': False}, {'question': 'في أي مدينة و ولاية نشأت بيونسيه؟', 'id': 4, 'answers': [{'text': 'هيوستن بولاية تكساس،', 'answer_start': 88}], 'is_impossible': False}, {'question': 'في أي عقد أصبحت بيونسي مشهورة؟', 'id': 5, 'answers': [{'text': 'أواخر التسعينات', 'answer_start': 223}], 'is_impossible': False}, {'question': 'في أي مجموعة R&B كانت هي المغنية الرئيسية؟', 'id': 6, 'answers': [{'text': 'دستنيز تشايلد', 'answer_start': 290}], 'is_impossible': False}, {'question': 'ما الألبوم الذي جعلها فنانة معروفة عالميا؟', 'id': 7, 'ans

# Read


In [3]:
def read_data(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa['answers']:
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    return contexts, questions, answers


In [4]:
contexts, questions, answers = read_data('AQAD-master/AQQAD 1.0/FINAL_AAQAD-v1.0.json')
#splitting the data into train and test
from sklearn.model_selection import train_test_split
train_contexts, val_contexts, train_questions, val_questions, train_answers, val_answers = train_test_split(contexts, questions, answers, test_size=0.1, random_state=42)

In [5]:
display(train_questions[0:3])
display(train_answers[0:3])
display(train_contexts[0:3])

['ما الذي تم بناء استاد فرنسا من أجله؟',
 'كيف ألقاب الجماعة درع لديه أرسنال؟',
 'في أي اتجاه يُنشر الخشب في كثير من الأحيان بحيث تظهر عقدة كدائرة صلبة تتدفق الحبوب حولها؟']

[{'text': 'كأس العالم لكرة القدم', 'answer_start': 187},
 {'text': '2014،', 'answer_start': 919},
 {'text': 'الألواح بطول 366 سم وبعرض', 'answer_start': 9}]

['يوجد في باريس العديد من الملاعب المخصصة لمختلف أنواع الرياضات. يعدّ ملعب فرنسا الذي يتسع لأكثر من 80 ألف متفرج أكبر ملاعب البلاد، وكان قد بني هذا الملعب الواقع في منطقة سان دينس لاستضافة كأس العالم لكرة القدم 1998 والذي حازت عليه فرنسا للمرة الأولى في تاريخها. يستخدم الملعب لممارسة كرة القدم، والرجبي، وألعاب القوى. يستضيف الملعب مباريات منتخب فرنسا الوطني للرجبي سنوياً في بطولة الأمم الستة. وكذلك يستضيف مباريات منتخب فرنسا لكرة القدم الودية وتصفيات البطولات الكبرى. بالإضافة إلى نادي باريس سان جيرمان، تملك المدينة نوادي كرة قدم أخرى مثل: نادي باريس، والنجم الأحمر ونادي فرنسا.\n',
 'شهد عام 2006 نجاحا للأرسنال على المستوى الأوروبي حيث تأهل للمرة الأولى في تاريخيه إلى المباراة النهائية لدوري أبطال أوروبا التي أقيمت في العاصمة الفرنسية باريس وجمعت الأرسنال مع نادي برشلونة الإسباني وكان الأرسنال قريبا من تحقيق لقبه الأوروبي الأول بعد أن أنهى الشوط الأول بتقدمه بهدف سجله مدافعه الإنجليزي سول كامبل. لكن أحلامهم في الحصول على اللقب تبددت بعد أن تمكن النادي الإسباني من قلب النتيجة لصالحه بتسج

In [6]:
display(val_questions[0:3])
display(val_answers[0:3])
display(val_contexts[0:3])

['ما هي البكتيريا التي تسبب عدوى السل؟',
 'ما هي البكتيريا التي تسبب مرض الحمى التي رصدت في جبال روكي؟',
 'كم من السكان الأصليين بالقرب من ولاية ماساتشوستس لقوا حتفهم بسبب الجدري في الوباء بين عامي 1617 و 1619؟']

[{'text': 'السل', 'answer_start': 89},
 {'text': 'أنواع', 'answer_start': 706},
 {'text': '30% من الأمريكيين الأصليين القاطنين في', 'answer_start': 512}]

['من المُقدّر أن حوالي 5 - 10٪ من غير المُصابين بفيروس نقص المناعة البشرية والمصابين بعدوى السل سيصبح مرضهم نشطًا خلال حياتهم وفي المقابل، فإن 30٪ من المُصابين بفيروس نقص المناعة البشرية سيصبح مرضهم نشطًا. قد يُصيب السل أي جزء من الجسم، ولكن إصابته للرئتين تُعتبر الأكثر شيوعًا (وهو ما يُعرف باسم السل الرئوي). يحدث السل خارج الرئوي عندما يتطور مرض السل خارج الرئتين. قد يتشارك السل خارج الرئوي مع السل الرئوي أيضًا  تشمل العلامات والأعراض العامة الحمى والقشعريرة والتعرق الليلي وفقدان الشهية وفقدان الوزن والتعب،  وقد يحدث تعجّر الأصابع الواضح أيضًا.\n',
 'يمتلك كل نوع من الأمراض مميزات خاصة به تمكنه من التفاعل مع مستقبلاته في الجسم البشري. فبعض الكائنات الحية مثل المكورات العقدية يمكن أن تسبب الالتهابات الجلدية، والالتهاب الرئوي، والسحايا، وتسمم الدم حيث ينتج صدمة فتتوسع وتتضخم الأوعية مما يسبب الموت. مع ذلك فإن هذه الكائنات الحية هي أيضا جزء من الإنسان، وعادةً ما تتواجد على الجلد أو في الأنف من دون أن تسبب أي مرض على الإطلاق. وهناك كائنات أخرى دائماً تسبب الأمراض لدى البشر مثل الريكتسية و

# Prepare

In [7]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    # this means the answer is off by 'n' tokens
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

In [8]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [9]:
train_answers = list(filter(lambda item: list(item.keys()) == ['text', 'answer_start', 'answer_end'], train_answers))

\check duplicates

In [10]:
len(train_answers)

10308

In [11]:
pd.DataFrame(train_answers).drop_duplicates(subset=['text','answer_start','answer_end'])

,text,answer_start,answer_end
0,كأس العالم لكرة القدم,187,208
1,2014،,919,924
2,الألواح بطول 366 سم وبعرض,9,34
3,2012,424,428
4,لامع أبيض,10,19
...,...,...,...
10300,أكثر من 700000 $,22,38
10301,فلسطين،,48,55
10302,بالقرب من مورغانتون الحالية,123,150
10303,مناعي,61,66


In [12]:
train_answers = [dict(t) for t in {tuple(d.items()) for d in train_answers}]

In [13]:
#loop over train answers and train contexts and make sure that they match
def check_answers_contexts_match(answers, contexts, questions, show_anamolies=False):
    
    answers_without_answer_end_key =0
    non_matching_answers=0
    for answer, context, question in zip(answers, contexts, questions):
        
        #check if context has key answer_end and if not 
        if 'answer_end' not in answer.keys():
            answers_without_answer_end_key +=1
            if show_anamolies:
                print(question)
                print(answer)
                print(context[answer['answer_start']:answer['answer_start']+ len(answer['text'])+1])
            continue 

        #check if answer matches context
        if context[answer['answer_start']:answer['answer_end']] != answer['text']:
            non_matching_answers +=1
    display('answers without answer_end key:', answers_without_answer_end_key,'non matching answers:', non_matching_answers)
    
check_answers_contexts_match(train_answers, train_contexts, train_questions)
check_answers_contexts_match(val_answers, val_contexts, val_questions)

'answers without answer_end key:'

0

'non matching answers:'

9715

'answers without answer_end key:'

6

'non matching answers:'

0

# Encode

In [14]:
from transformers import DistilBertTokenizerFast
disilbert_tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [15]:
%pip install arabert
from transformers import AutoTokenizer
# from arabert.preprocess_arabert import never_split_tokens, preprocess
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabert", do_lower_case=False, do_basic_tokenize=True)


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [16]:
def show_encoding_of(sentence, tokenizer):
    encoding = tokenizer(sentence)
    print(sentence)
    print(encoding)
    print('length of encoding:', len(encoding['input_ids']))
    for i in encoding['input_ids']:
        #print the decoding of it
        print(i,':',tokenizer.decode([i]))

In [17]:
test_text = "الألواح الخشبية تعالج بعد ذلك في أفران للتحميص حتى لا تتأثر مستقبلاً بتغييرات درجات الحرارة أو الرطوبة الموجودة في الجو"
show_encoding_of(test_text, arabert_tokenizer)

الألواح الخشبية تعالج بعد ذلك في أفران للتحميص حتى لا تتأثر مستقبلاً بتغييرات درجات الحرارة أو الرطوبة الموجودة في الجو
{'input_ids': [29756, 445, 27375, 7800, 445, 27960, 1007, 980, 33864, 3161, 4312, 781, 30353, 113, 1001, 27671, 8259, 3842, 818, 32967, 29759, 91, 53212, 7797, 4179, 7797, 445, 7874, 7804, 980, 402, 445, 988, 28626, 980, 445, 54052, 980, 781, 445, 7866, 29758], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
length of encoding: 42
29756 : [CLS]
445 : ال
27375 : ##ألو
7800 : ##اح
445 : ال
27960 : ##خشب
1007 : ##ي
980 : ##ة
33864 : تعالج
3161 : بعد
4312 : ذلك
781 : في
30353 : أفران
113 : ل
1001 : ##ل
27671 : ##تحم
8259 : ##يص
3842 : حتى
818 : لا
32967 : تتأثر
29759 : [UNK]
91 : ب
53212 : ##تغيير
7797 : ##ات
4179 : درج
7797 : ##ات


In [18]:
show_encoding_of(test_text, disilbert_tokenizer)

الألواح الخشبية تعالج بعد ذلك في أفران للتحميص حتى لا تتأثر مستقبلاً بتغييرات درجات الحرارة أو الرطوبة الموجودة في الجو
{'input_ids': [101, 1270, 23673, 25573, 23673, 29836, 25573, 29820, 1270, 23673, 29821, 29825, 29816, 14498, 19433, 1273, 29830, 25573, 23673, 29819, 1271, 29830, 15394, 1279, 23673, 29835, 1291, 14498, 1270, 29833, 17149, 18511, 1294, 23673, 29817, 29820, 22192, 14498, 29826, 1276, 29817, 29837, 1294, 25573, 1273, 29817, 25573, 29818, 17149, 1295, 29824, 29817, 29834, 29816, 23673, 25573, 1271, 29817, 29831, 14498, 14498, 17149, 25573, 29817, 1278, 17149, 29819, 25573, 29817, 1270, 23673, 29820, 17149, 25573, 17149, 19433, 1270, 29836, 1270, 23673, 17149, 29828, 29836, 29816, 19433, 1270, 23673, 22192, 29836, 29819, 29836, 15394, 19433, 1291, 14498, 1270, 23673, 29819, 29836, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [19]:
tokenizer = arabert_tokenizer
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [20]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        
        # old
        # start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        # end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
        # Mine mah, lol I am sure it must work
        start_positions.append(encodings.char_to_token(i, answers[i].get('answer_start', None)))
        end_positions.append(encodings.char_to_token(i, answers[i].get('answer_end', None)))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        print(end_positions[-1])
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, train_answers)

49
5
53
133
25
None
38
127
None
None
32
None
76
None
81
None
None
49
116
None
None
13
15
177
None
None
4
32
53
27
42
None
None
22
42
None
106
59
6
43
111
44
None
None
None
None
138
None
37
100
4
8
5
77
280
None
36
10
69
None
50
46
None
74
None
None
75
None
None
None
40
None
108
6
43
None
2
None
37
None
None
63
60
55
103
92
7
22
None
101
42
31
None
155
67
177
97
58
5
48
12
14
141
240
144
6
13
14
107
None
56
173
None
188
None
121
None
None
33
None
65
None
178
30
15
None
84
32
101
31
150
37
183
9
183
148
169
57
None
33
None
31
138
35
6
41
50
None
54
None
None
None
9
130
110
110
None
95
144
126
104
64
None
None
63
241
None
83
None
None
58
9
None
71
4
71
20
103
191
2
None
82
49
None
None
77
46
62
172
133
None
36
64
None
None
90
93
None
None
36
87
257
241
11
228
35
130
26
15
90
46
None
None
21
198
None
None
None
25
109
None
13
None
43
69
None
None
29
2
50
None
173
180
None
36
14
None
338
151
125
None
64
118
None
88
15
156
38
41
45
None
19
28
105
83
81
102
90
None
102
None
None
None
None
None

In [21]:
train_encodings['start_positions'][0:100]

[45,
 3,
 46,
 129,
 18,
 86,
 26,
 72,
 130,
 512,
 28,
 512,
 73,
 512,
 79,
 512,
 76,
 45,
 25,
 512,
 512,
 12,
 11,
 146,
 512,
 512,
 3,
 29,
 512,
 20,
 41,
 120,
 14,
 18,
 39,
 87,
 512,
 58,
 3,
 5,
 109,
 43,
 512,
 512,
 102,
 512,
 116,
 512,
 37,
 96,
 4,
 5,
 2,
 75,
 512,
 512,
 34,
 7,
 48,
 512,
 33,
 512,
 512,
 53,
 3,
 512,
 68,
 121,
 512,
 512,
 32,
 512,
 101,
 4,
 512,
 512,
 1,
 512,
 33,
 12,
 512,
 63,
 1,
 512,
 100,
 84,
 6,
 20,
 512,
 98,
 40,
 28,
 187,
 153,
 64,
 174,
 96,
 39,
 2,
 44]